# K-Label of States

Goals:

1. Understand how to identify k-labels of FCI states
2. Learn fundamentals of MPS with pyblock3(renormalizer), then Block2
3. Identify k-labels for 50-site Hubbard Excited States

In [ ]:
from pyblock3.hamiltonian import Hamiltonian
from pyblock3.fcidump import FCIDUMP
from pyblock3.algebra.mpe import MPE
from pyscf.fci import cistring, FCI
import numpy as np

import sys
sys.path.append("..")
from dmet.model import hubbard

def build_hubbard(u=6, t=1, n=6, cutoff=1E-9):
    fcidump = FCIDUMP(pg='c1', n_sites=n, n_elec=n, twos=0, ipg=0, orb_sym=[0] * n)
    hamil = Hamiltonian(fcidump, flat=False)

    def generate_terms(n_sites, c, d):
        for i in range(0, n_sites):
            for s in [0, 1]:
                    yield t * c[i, s] * d[(i-1)%n_sites, s]
                    yield t * c[i, s] * d[(i+1)%n_sites, s]
            yield u * (c[i, 0] * c[i, 1] * d[i, 1] * d[i, 0])

    return hamil, hamil.build_mpo(generate_terms, cutoff=cutoff).to_sparse()

hamil, mpo = build_hubbard()

mps = hamil.build_mps(250)

In [ ]:
# n_start, twos_start, n, twos


def mps2fci(norb, nelec, mps):

    qlabeldicts = [dict(map(reversed, enumerate([(
    blk.q_labels[0].n, blk.q_labels[0].twos, 
    blk.q_labels[1].n, blk.q_labels[1].twos) for blk in tensor.blocks])))
    for tensor in mps.tensors]

    neleca, nelecb = nelec
    occas = cistring._gen_occslst(range(norb), neleca)
    occbs = cistring._gen_occslst(range(norb), nelecb)

    nstra = cistring.num_strings(norb, neleca)
    nstrb = cistring.num_strings(norb, nelecb)

    ci = np.zeros((nstra, nstrb))

    for i in range(nstra):
        occa = occas[i]
        for j in range(nstrb):
            occb = occbs[j]

            nlst = np.zeros(norb)
            twoslst = np.zeros(norb)
            for o in occa: nlst[o] += 1; twoslst[o] += 1
            for o in occb: nlst[o] += 1; twoslst[o] -= 1

            def _mul(a, b):
                if type(a) == float: return a * b
                else: return np.einsum("abc,cbe->abe", a, b)
        
            res = 1.
            n = 0
            twos = 0
            for idx in range(norb):
                qlabel = (n, twos, nlst[idx], twoslst[idx])
                blockidx = qlabeldicts[idx][qlabel]
                tensor = np.asarray(mps.tensors[idx].blocks[blockidx])

                res = _mul(res, tensor)
                n += nlst[idx]; twos += twoslst[idx]
                
            res = np.linalg.norm(res)   # 1*1*1 tensor -> number
            ci[i, j] = res

    return ci

In [14]:
from pyscf.fci import cistring, FCI
from pyscf.scf import hf
import numpy as np
import sys
sys.path.append("..")
from dmet.model import hubbard
from pyscf.fci import fci_slow

L = 6; U = 6

mol = hubbard.HubbardMole(L, 1, 1, U)
mol.nelectron = L
mf = hubbard.RHF_SpinHubbard(mol)
mf.pbc = True
mf.kernel()
h1e = mf.get_hcore()
eri = mf._eri

e, h, p, c = fci_slow.kernel(h1e, eri, L, L, nroots=12)
print(e)
print(len(c))

na = len(list(cistring.make_strings(range(L), L//2)))
nb = len(list(cistring.make_strings(range(L), L//2)))

def calc_trans(ci0, norb, nelec):
    neleca, nelecb = nelec
    threshold = 2**norb

    str_a = list(cistring.make_strings(range(norb), neleca))
    str_b = list(cistring.make_strings(range(norb), nelecb))
    res = 0

    for i, stra in enumerate(str_a):
        changed_stra = 2*stra
        if changed_stra >= threshold: changed_stra = changed_stra%threshold+1
        changed_i = str_a.index(changed_stra)

        for j, strb in enumerate(str_b):
            changed_strb = 2*strb
            if changed_strb >= threshold: changed_strb = changed_strb%threshold+1
            changed_j = str_b.index(changed_strb)

            res += ci0[i,j] * ci0[changed_i, changed_j]

    return res

for ci in c:
    print(calc_trans(ci.reshape((na, nb)), L, (L//2,L//2)))

converged SCF energy = 0.999999999999998


Initialize <dmet.model.hubbard.HubbardMole object at 0x7fcc273beb80> in <dmet.model.hubbard.RHF_SpinHubbard object at 0x7fcc273be580>


[-2.64851756 -2.16342845 -1.83805413 -1.72630329 -1.72630329 -1.52638607
 -1.52638607 -1.25456083 -1.25456039 -1.21874398 -0.92434387 -0.92432744]
12
1.0000000000000007
-0.9999999999999946
-0.9999999999999983
0.4999999999999815
0.49999999985483345
-0.4999999999922667
-0.4999999998872824
-0.5000000000000003
-0.5000003654558964
0.999999992457074
0.4999996295033995
0.4995923565462643


In [15]:
#Block2
import numpy as np
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

L = 6
N = 6
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

t = 1
U = 6

b = driver.expr_builder()

# hopping term
b.add_term("cd",
    np.array([[[i, (i+1)%L], [(i+1)%L, i]] for i in range(L)]).flatten(),
    [-t] * 2 * (L))
b.add_term("CD",
    np.array([[[i, (i+1)%L], [(i+1)%L, i]] for i in range(L)]).flatten(),
    [-t] * 2 * (L))

# onsite term
b.add_term("cdCD",
    np.array([[i, ] * 4 for i in range(L)]).flatten(),
    [U] * L)

mpo = driver.get_mpo(b.finalize(), iprint=2, add_ident=False)
ket = driver.get_random_mps(tag="KET", bond_dim=400, nroots=12)

def run_dmrg(driver, mpo):
    bond_dims = [200] * 4 + [400] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(
        mpo,
        ket,
        n_sweeps=25,
        bond_dims=bond_dims,
        noises=noises,
        thrds=thrds,
        iprint=2,
    )

energies = run_dmrg(driver, mpo)
print(energies)



Build MPO | Nsites =     6 | Nterms =         30 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     6 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.007
 Site =     1 /     6 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.7500 Tmvc = 0.000 T = 0.003
 Site =     2 /     6 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8500 Tmvc = 0.000 T = 0.004
 Site =     3 /     6 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8500 Tmvc = 0.000 T = 0.008
 Site =     4 /     6 .. Mmpo =     6 DW = 0.00e+00 NNZ =       15 SPT = 0.7500 Tmvc = 0.000 T = 0.002
 Site =     5 /     6 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.005
Ttotal =      0.029 Tmvc-total = 0.000 MPO bond dimension =    10 MaxDW = 0.00e+00
NNZ =           72 SIZE =          332 SPT = 0.7831

Rank =     0 Ttotal =      0.061 MPO method = FastBipartite bond dimension =      10 NNZ =           72 SIZE =          332 SPT = 0.7831
Environment initi

In [16]:
from pyblock3.block2.io import MPSTools
from pyblock3.algebra.core import SparseTensor

def calc_trans_mps(mps, nelec=None):

    t = mps.tensors
    norb = len(t)
    if nelec is None: nelec = norb

    res = SparseTensor._tensordot(t[0], t[1], axes=([2],[0]))
    res = SparseTensor._tensordot(res, t[0], axes=([2],[1]))
    res = SparseTensor._tensordot(res, t[2], axes=([2],[0]))
    res = SparseTensor._tensordot(res, t[1], axes=([3,4],[0,1]))
    for i in range(2, norb-1):
        res = SparseTensor._tensordot(res, t[i+1], axes=([3],[0]))
        res = SparseTensor._tensordot(res, t[i], axes=([3,4],[0,1]))

    # Phase Factor
    t2 = mps.copy().tensors
    t2[norb-1].blocks[1] *= -1**(nelec-1)
    t2[norb-1].blocks[2] *= -1**(nelec-1)

    res = SparseTensor._tensordot(res, t2[norb-1], axes=([4,1,3],[0,1,2]))
    return res.blocks[0].reshape(-1)[0]

# multi-mps

for i in range(12):
    theket = driver.split_mps(ket, iroot=i, tag="EXT-{}".format(i+1))
    dasket = driver.adjust_mps(theket, dot=1)[0]
    pyket = MPSTools.from_block2(dasket)
    print(calc_trans_mps(pyket))

0.9999999999999394
-0.9999999999998944
-0.9999999999996565
0.49999999999988376
0.49999999999961947
-0.499999999999884
-0.49999999999964645
-0.4999999999997497
-0.49999999999965283
0.9999999999996223
0.49999999999841516
0.49999999999423467


In [8]:
from pyblock3.block2.io import MPSTools
from pyblock3.algebra.core import SparseTensor

def calc_trans_mps(mps, nelec=None):

    t = mps.tensors
    norb = len(t)
    if nelec is None: nelec = norb

    res = SparseTensor._tensordot(t[0], t[1], axes=([2],[0]))
    res = SparseTensor._tensordot(res, t[0], axes=([2],[1]))
    res = SparseTensor._tensordot(res, t[2], axes=([2],[0]))
    res = SparseTensor._tensordot(res, t[1], axes=([3,4],[0,1]))
    for i in range(2, norb-1):
        res = SparseTensor._tensordot(res, t[i+1], axes=([3],[0]))
        res = SparseTensor._tensordot(res, t[i], axes=([3,4],[0,1]))

    # Phase Factor
    t2 = mps.copy().tensors
    t2[norb-1].blocks[1] *= -1**(nelec-1)
    t2[norb-1].blocks[2] *= -1**(nelec-1)

    res = SparseTensor._tensordot(res, t2[norb-1], axes=([4,1,3],[0,1,2]))
    return res.blocks[0].reshape(-1)[0]

# multi-mps

for i in range(12):
    theket = driver.split_mps(ket, iroot=i, tag="EXT-{}".format(i+1))
    dasket = driver.adjust_mps(theket, dot=1)[0]
    pyket = MPSTools.from_block2(dasket)
    print(calc_trans_mps(pyket))

0.9999980489737293
-0.9999962038428925
-0.9999968268373105
0.8090143757359658
0.8090137241568754
-0.8090142739822985
-0.809013723298151
0.999995245733048
-0.8090142709927151
-0.8090138323244593
0.30901577165698524
0.30901568636397503


In [12]:
np.cos(3*np.pi/10)

0.5877852522924731

In [ ]:


pyket = MPSTools.from_block2(ket)
print('MPS overlap using numpy arrays =', pyket @ pyket)
print(pyket[0])

In [ ]:
def my_calc_ovlp(mps):

    t = mps.tensors.copy()
    norb = len(t)
    res = SparseTensor._tensordot(t[0], t[0], axes=([1], [1]))
    res = SparseTensor._tensordot(res, t[1], axes=([1],[0]))
    res = SparseTensor._tensordot(res, t[1], axes=((2,3),(0,1)))
    for i in range(2, norb):
        res2 = SparseTensor._tensordot(res, t[i], axes=([2],[0]))
        res = SparseTensor._tensordot(res2, t[i], axes=((2,3),(0,1)))
    return float(res[0].reshape(-1))

my_calc_ovlp(pyket)

In [ ]:
def my_calc_ovlp2(mps):
    t = mps.tensors
    norb = len(t)
    res = SparseTensor._tensordot(t[0], t[1], axes=([2],[0]))
    res = SparseTensor._tensordot(res, t[1], axes=([2],[1]))
    res = SparseTensor._tensordot(res, t[2], axes=([2],[0]))
    res = SparseTensor._tensordot(res, t[2], axes=([3,4],[0,1]))
    for i in range(3, norb):
        res = SparseTensor._tensordot(res, t[i], axes=([3],[0]))
        res = SparseTensor._tensordot(res, t[i], axes=([3,4],[0,1]))
    res = SparseTensor._tensordot(res, t[0], axes=([1,2],[1,2]))
    return res
my_calc_ovlp2(pyket)


In [ ]:

def calc_trans_mps(mps):

    t = mps.tensors
    norb = len(t)

    res = SparseTensor._tensordot(t[0], t[1], axes=([2],[0]))
    res = SparseTensor._tensordot(res, t[0], axes=([2],[1]))
    res = SparseTensor._tensordot(res, t[2], axes=([2],[0]))
    res = SparseTensor._tensordot(res, t[1], axes=([3,4],[0,1]))
    for i in range(2, norb-1):
        res = SparseTensor._tensordot(res, t[i+1], axes=([3],[0]))
        res = SparseTensor._tensordot(res, t[i], axes=([3,4],[0,1]))

    # Phase Factor
    t2 = mps.copy().tensors
    t2[norb-1].blocks[1] *= -1
    t2[norb-1].blocks[2] *= -1

    res = SparseTensor._tensordot(res, t2[norb-1], axes=([4,1,3],[0,1,2]))
    return res.blocks[0].reshape(-1)[0]

calc_trans_mps(pyket)

$$

| \sigma_1 \sigma_2 \cdots \sigma_N \rangle = \hat a^\dagger_{\sigma_N} \cdots \hat a^\dagger_{\sigma_1} | 0 \rangle 

$$

in which

$$

\hat a^\dagger_{\uparrow \downarrow} = \hat a^\dagger_{\beta} \hat a^\dagger_{\alpha}

$$

In [ ]:
# n_start, twos_start, n, twos

# Def of site : < vac | 1_a, 1_b 2_b 3_a 4_b 5_a 5_b 6_a ...


def mps2fci(norb, nelec, mps):

    qlabeldicts = [dict(map(reversed, enumerate([(
    blk.q_labels[0].n, blk.q_labels[0].twos, 
    blk.q_labels[1].n, blk.q_labels[1].twos) for blk in tensor.blocks])))
    for tensor in mps.tensors]

    def _pf_change(occa, occb):

        nperm = 0
        for a in occa:
            for b in occb:
                if b < a: nperm += 1
        return (-1) ** nperm


    neleca, nelecb = nelec
    occas = cistring._gen_occslst(range(norb), neleca)
    occbs = cistring._gen_occslst(range(norb), nelecb)

    nstra = cistring.num_strings(norb, neleca)
    nstrb = cistring.num_strings(norb, nelecb)

    ci = np.zeros((nstra, nstrb))

    for i in range(nstra):
        occa = occas[i]
        for j in range(nstrb):
            occb = occbs[j]

            nlst = np.zeros(norb)
            twoslst = np.zeros(norb)
            for o in occa: nlst[o] += 1; twoslst[o] += 1
            for o in occb: nlst[o] += 1; twoslst[o] -= 1

            def _mul(a, b):
                if type(a) == float: return a * b
                else: return np.einsum("abc,cbe->abe", a, b)
            sign = _pf_change(occa, occb)
            res = 1.
            n = 0
            twos = 0
            for idx in range(norb):
                qlabel = (n, twos, nlst[idx], twoslst[idx])
                blockidx = qlabeldicts[idx][qlabel]
                tensor = np.asarray(mps.tensors[idx].blocks[blockidx])

                res = _mul(res, tensor)
                n += nlst[idx]; twos += twoslst[idx]
                
            res = res[0,0,0] * sign  # 1*1*1 tensor -> number * phase_factor
            ci[i, j] = res

    return ci

In [ ]:
ci = mps2fci(6, (3,3), pyket)

print(ci[0,1]-ci[3,6])

In [ ]:
ci = mps2fci(6, (3,3), pyket)

import sys
sys.path.append("..")

from dmet.model import hubbard
from pyscf.fci import direct_spin1

L = 6; U = 6; t = 1

mol = hubbard.HubbardMole(lx=L, ly=1, t=t, U=U)
mol.nelectron = L
mf = hubbard.RHF_SpinHubbard(mol)
mf.pbc = True
mf.kernel()

fcisolver = FCI(mf)
e_fci = fcisolver.kernel(nroots=2)[0]
print(e_fci)
h1e = mf.get_hcore()
eri = mf._eri

print(direct_spin1.energy(h1e, eri, ci, L, L))

In [ ]:
print(calc_trans(ci.reshape((na, nb)), L, (L//2,L//2)))

In [ ]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2.algebra.io import MPSTools, MPOTools

from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, add_ident=False, iprint=1)
print(mpo)
ket = driver.get_random_mps(tag="GS", bond_dim=250, nroots=1)
bond_dims = [250] * 4 + [500] * 4
noises = [1e-4] * 4 + [1e-5] * 4 + [0]
thrds = [1e-10] * 8
energies = driver.dmrg(mpo, ket, n_sweeps=5, bond_dims=bond_dims,
    noises=noises, thrds=thrds, iprint=1)
print('DMRG energy = %20.15f' % energies)

ket = driver.adjust_mps(ket, dot=1)[0]
print(ket.n_sites, ket.canonical_form)

pyket = MPSTools.from_block2(ket)
print('MPS tensor at site 1 =\n', pyket.tensors[1])

pympo = MPOTools.from_block2(mpo.prim_mpo)
print('MPO tensor at site 1 =\n', pympo.tensors[1])

print('MPS overlap using numpy arrays =', pyket @ pyket)
print('Energy expectation using numpy arrays =', pyket @ (pympo @ pyket))

In [ ]:
# Gauge factor?

# |__|__|__|_o_|